## Import statements

In [1]:
import h5py
import sys
sys.path.append('..')
from modules.configfile import config
import matplotlib.pyplot as plt
import cPickle as pickle
import numpy as np
import random
from random import shuffle
random.seed(1337)
np.random.seed(1337)

/home/anmol/.virtualenvs/brats/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Open mean and variance file

In [2]:
mean_var = pickle.load(open(config['saveMeanVarCombinedData'], 'rb'))

In [3]:
mean_var

{'mn': [101.05125, 104.49648, 111.35362, 70.27612],
 'var': [299264.5, 362103.22, 317921.5, 319424.38]}

## Open new database with cropped images

In [4]:
hdf5_file = h5py.File(config['hdf5_combined'], mode='r')

In [5]:
hdf5_file_g = hdf5_file['combined']

In [6]:
hdf5_file_g['training_data'].shape

(285, 4, 240, 240, 155)

In [7]:
def apply_mean_std(im, mean_var):
    # expects a dictionary of means and VARIANCES, NOT STD
    for m in range(0,4):
        if len(np.shape(m)) > 4:
            im[:,m,...] = (im[:,m,...] - mean_var['mn'][m]) / np.sqrt(mean_var['var'][m])
        else:
            im[m,...] = (im[m,...] - mean_var['mn'][m]) / np.sqrt(mean_var['var'][m])
            
    return im

## Get all the HGG data

In [8]:
# training_data_hgg = hdf5_file_g['training_data_hgg']
# training_data_segmasks_hgg = hdf5_file_g['training_data_segmasks_hgg']

# Start the Iteration here. This is the "Epoch"

## Generate random access order

In [9]:
indices = list(range(0, hdf5_file_g['training_data'].shape[0]))
shuffle(indices)

## Split indices into training and testing

In [10]:
train_end = int((len(indices) * config['data_split']['train']) / 100.0)
train_indices = indices[0:train_end]
test_indices = indices[train_end:]

In [11]:
train_indices[0:5]

[277, 67, 207, 140, 148]

#### In the train_seg.py, this is the original indices:

[255, 219, 107, 230, 46, 165, 103, 151, 176]

In [12]:
test_indices

[255, 219, 107, 230, 46, 165, 103, 151, 176]

# Create a test set which is independent of any patient that was used in multimodal-synthesis model training

The multimodal-synthesis model was trained with indices 0-54 from the Combined HDF5 file. Do not use any data from this range as your test data. 

In [13]:
new_test_ind = [x for x in test_indices if x > 54]

In [14]:
new_test_ind

[255, 219, 107, 230, 165, 103, 151, 176]

### test_data contains the test set which neither multimodal-synthesis model not BRATS model has seen. So it will be a fair comparison

In [15]:
test_data = hdf5_file_g['training_data'][sorted(new_test_ind)]
test_data_masks = hdf5_file_g['training_data_segmasks'][sorted(new_test_ind)]

In [16]:
test_data.shape

(8, 4, 240, 240, 155)

In [17]:
test_data_masks.shape

(8, 240, 240, 155)

### Define Dice Coefficient in Numpy

In [18]:
def dice(im1, im2, empty_score=1.0):
    im1 = np.asarray(im1).astype(np.bool)
    im2 = np.asarray(im2).astype(np.bool)

    if im1.shape != im2.shape:
        raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")

    im_sum = im1.sum() + im2.sum()
    if im_sum == 0:
        return empty_score

    # Compute Dice coefficient
    intersection = np.logical_and(im1, im2)

    return 2. * intersection.sum() / im_sum

# Load BRATS model for segmentation of the test_data patients

In [ ]:
from modules.training_helpers import standardize
from modules.vizhelpercode import viewArbitraryVolume
import importlib

In [ ]:
defmodelfile = 'isensee'
model_name = '/home/anmol/mounts/cedar-rm/scratch/asa224/model-staging/isensee_da_noanneal64--0.64.h5'

In [ ]:
modeldefmodule = importlib.import_module('defmodel.' + defmodelfile, package=None)

In [ ]:
custom_objs = modeldefmodule.custom_loss()
model = modeldefmodule.open_model_with_hyper_and_history(name=model_name, custom_obj=custom_objs, load_model_only=True)

### Test the model on test_data

In [ ]:
td_shape = test_data.shape
test_data_pred = np.zeros((td_shape[0], 3, td_shape[2], td_shape[3], td_shape[4]))

for i in range(0, test_data.shape[0]):
    print('Patient {}'.format(i+1))
    pat_volume = test_data[i]
    print('Standardizing..')
    pat_volume = standardize(pat_volume, applyToTest=mean_var)

    curr_shape = list(pat_volume.shape)
    curr_shape.insert(0, 1) # insert 1 at index 0 to make reshaping easy

    pat_volume = pat_volume.reshape(curr_shape)

    # SUPER HACK WAY TO CHANGE VOLUME COMPATIBILITY WITH ISENSEE MODEL. MAKE 155 = 160
    new_pat_volume = np.zeros((1, 4, 240, 240, 160))
    new_pat_volume[:, :, :, :, 0:155] = pat_volume

    print('Starting prediction..')
    # predict using the whole volume
    pred = model.predict(new_pat_volume)

    # get back the main volume and strip the padding
    pred = pred[:,:,:,:,0:155]

    assert pred.shape == (1,3,240,240,155)

    print('Adding predicted volume to test_data_pred store..')
    # we use the batch size = 1 for prediction, so the first one.
    test_data_pred[i] = pred[0]
    viewArbitraryVolume(test_data_pred[i], slice_idx=3)

### Save test_data_pred numpy array

# Load multimodal-synthesis model trained with T1-T2 as input and T2FLAIR as output

In [19]:
import os
os.environ["KERAS_BACKEND"] = "theano"

In [20]:
from multimodal.loader_multimodal import Data
from multimodal.runner import Experiment

Using Theano backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
../multimodal/model.py:5: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/anmol/.virtualenvs/brats/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/anmol/.virtualenvs/brats/local/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/anmol/.virtualenvs/brats/lo

# FIX THIS: The data loading has to be handled in a way that multimodal-synthesis does, in the form of slices. 

In [28]:
model_path = '/home/anmol/mounts/cedar-rm/rrg_proj_dir/multimodal_brain_synthesis/RESULTS/split57_no_ST_LGG_only_T1_T2-T2FLAIR/'
data_dir = '/home/anmol/projects/multimodal_brain_synthesis/npz_BRATS/'
data = Data(data_dir, dataset='BRATS', trim_and_downsample=False, modalities_to_load=['T1', 'T2', 'T2FLAIR'],
            normalize_volumes=False)
data.load()

Loading T1
Loading T2
Loading T2FLAIR


In [32]:
data.T1[2].shape

(155, 240, 240)

In [ ]:
input_modalities = ['T1', 'T2']
output_weights = {'T2FLAIR': 1.0, 'concat': 1.0}
print('Creating experiment...')
exp = Experiment(input_modalities, output_weights, './RESULTS', data=None, latent_dim=16, spatial_transformer=False)

load_input_modalities = ['T1', 'T2']
load_output_modalities = 'T2FLAIR'

print('Loading partial model')
exp.load_partial_model(folder=model_path, model_name='model', input_modalities=load_input_modalities,
                       output_modality=load_output_modalities)

### Prepare T1 and T2 data for the synthesis model <br>
```
if 't1.' 
    i = 0
    seq_name = 't1'
elif 't2.' in imagefile:
    i = 1
    seq_name = 't2'
elif 't1ce.' in imagefile:
    i = 2
    seq_name = 't1ce'
elif 'flair.' in imagefile:
    i = 3
    seq_name = 'flair'
```

In [26]:
t1 = test_data[:,0,]
t2 = test_data[:,1,]

test_data_list = [t1, t2]

In [27]:
test_data_list[0].shape

(1240, 240, 240)

### Start prediction process

In [ ]:
Z = exp.mm.model.predict(test_data_list)

## Start the patch generation process

### Get the segmentation mask, find centroid and diameter of tumor region

In [ ]:
for idx in train_indices:
    patient_x_train = apply_mean_std(training_data_hgg[idx], mean_var)
    patient_y_train = training_data_segmasks_hgg[idx]
    break

## Let's call this (x, y, z)

<img alt="" height="279" src="https://www.med.upenn.edu/sbia/assets/user-content/BRATS_tasks.png" width="700">

The segmentations are combined to generate the final labels of the tumor sub-regions (Fig.D): edema (yellow), non-enhancing solid core (red), necrotic/cystic core (green), enhancing core (blue). (Figure taken from the BraTS IEEE TMI paper.)

### However, in the segmentation mask, the encoding is this - 

    1 for necrosis
    2 for edema
    3 for non-enhancing tumor
    4 for enhancing tumor
    0 for everything else

### To be able to weight the centre of mass correctly, the encoding needs to change.

In [ ]:
seg_reweighted = np.copy(patient_y_train)

In [ ]:
seg_reweighted[np.where(patient_y_train == 1)] = 10 # necrotic, the most inner region, has highest weight
seg_reweighted[np.where(patient_y_train == 4)] = 9 # enhancing
seg_reweighted[np.where(patient_y_train == 3)] = 8 # non-enhancing
seg_reweighted[np.where(patient_y_train == 2)] = 7 # edema

In [ ]:
m_x, m_y, m_z = center_of_mass(seg_reweighted)

## Now we need to find the extent of mass, in all directions - (x, y, z). This is the "standard deviation". 

Checked this using visualization

In [ ]:
def check_valid(patch_coords):
    patch_coords = [int(x) for x in patch_coords]
    xmin, xmax, ymin, ymax, zmin, zmax = patch_coords
    
    if xmin >=0 and xmax < config['size_after_cropping'][0] and \
                    ymin >=0 and ymax < config['size_after_cropping'][1] and \
                    zmin >=0 and zmax < config['size_after_cropping'][2]:
        return patch_coords
    else:
        return None

In [ ]:
x,y,z = np.where(patient_y_train > 0)
std_x = np.max(x) - np.min(x)
std_y = np.max(y) - np.min(y)
std_z = np.max(z) - np.min(z)

In [ ]:
k = 0
while k != None:
    patch_size_x, patch_size_y, patch_size_z = (60, 60, 60)
    std_scale = 1.8
    xmin, ymin, zmin = np.random.multivariate_normal(mean=[m_x, m_y, m_z], cov=np.diag(np.array([std_x, std_y, std_z])*std_scale))
    xmax = xmin + patch_size_x
    ymax = ymin + patch_size_y
    zmax = zmin + patch_size_z
    patch_coords = [xmin, xmax, ymin, ymax, zmin, zmax]
    t = check_valid(patch_coords)
    if t != None:
        k = None

# Consolidate Code

In [ ]:
def calculateCOM_STD(segmask):
    seg_reweighted = np.copy(segmask)

    # brute force way to make sure the COM calculation is weighted correctly. We need more
    # weight on necrotic region, than edema.
    seg_reweighted[np.where(segmask == 1)] = 10  # necrotic, the most inner region, has highest weight
    seg_reweighted[np.where(segmask == 4)] = 9  # enhancing
    seg_reweighted[np.where(segmask == 3)] = 8  # non-enhancing
    seg_reweighted[np.where(segmask == 2)] = 7  # edema

    # calculate COM
    m_x, m_y, m_z = center_of_mass(seg_reweighted)

    x, y, z = np.where(segmask > 0)
    std_x = np.max(x) - np.min(x)
    std_y = np.max(y) - np.min(y)
    std_z = np.max(z) - np.min(z)
    
    return [m_x, m_y, m_z], [std_x, std_y, std_z]

# TESTING

## VIsualize the patch in 3D

In [ ]:
from mayavi import mlab

In [ ]:
def createDense(bbox, im):
    box = np.zeros(im.shape)
    box[bbox[0]:bbox[1], bbox[2]:bbox[3], bbox[4]:bbox[5]] = 1
    return box

In [ ]:
# lets get a segmentation
seg = patient_y_train

dense_bbox = createDense(t, seg)

src = mlab.pipeline.scalar_field(seg)

src_bbox = mlab.pipeline.scalar_field(dense_bbox)
# mlab.pipeline.iso_surface(src, contours=[0, 1, 2, 3, 4], opacity=0.5)
mlab.pipeline.iso_surface(src, contours=[1], opacity=0.4, color=(0,1,0))
mlab.pipeline.iso_surface(src, contours=[2], opacity=0.4)
mlab.pipeline.iso_surface(src, contours=[3], opacity=0.4)
mlab.pipeline.iso_surface(src, contours=[4], opacity=0.4)
mlab.pipeline.iso_surface(src_bbox, contours=[1], opacity=0.2)
# mlab.pipeline.iso_surface(src, contours=[s.max()-0.1*s.ptp(), ],)
mlab.show()

## Dry run the patch generation pipeline and manually see the visualization

In [ ]:
import time

In [ ]:
count = 0
for idx in train_indices[3:]:
    patient_x_train = apply_mean_std(training_data_hgg[idx], mean_var)
    patient_y_train = training_data_segmasks_hgg[idx]
    
    seg_reweighted = np.copy(patient_y_train)
    
    seg_reweighted[np.where(patient_y_train == 1)] = 10 # necrotic, the most inner region, has highest weight
    seg_reweighted[np.where(patient_y_train == 4)] = 9 # enhancing
    seg_reweighted[np.where(patient_y_train == 3)] = 8 # non-enhancing
    seg_reweighted[np.where(patient_y_train == 2)] = 7 # edema
    
    m_x, m_y, m_z = center_of_mass(seg_reweighted)
    
    for _num in range(0, 10):
        k = 0
        while k != None:
            patch_size_x, patch_size_y, patch_size_z = (40, 40, 40)
            std_scale = 400
            xc, yc, zc = np.random.multivariate_normal(mean=[m_x, m_y, m_z], cov=np.diag(np.array([std_x, std_y, std_z])*std_scale))
            xmin = xc - patch_size_x
            xmax = xc + patch_size_x
            
            ymin = yc - patch_size_y
            ymax = yc + patch_size_y
            
            zmin = zc - patch_size_z
            zmax = zc + patch_size_z
            
#             xmax = xmin + patch_size_x
#             ymax = ymin + patch_size_y
#             zmax = zmin + patch_size_z
            patch_coords = [xmin, xmax, ymin, ymax, zmin, zmax]
            t = check_valid(patch_coords)
            if t != None:
                k = None

        # lets get a segmentation
        seg = patient_y_train

        dense_bbox = createDense(t, seg)

        src = mlab.pipeline.scalar_field(seg)

        src_bbox = mlab.pipeline.scalar_field(dense_bbox)
        # mlab.pipeline.iso_surface(src, contours=[0, 1, 2, 3, 4], opacity=0.5)
        mlab.pipeline.iso_surface(src, contours=[1], opacity=0.4, color=(0,1,0))
        mlab.pipeline.iso_surface(src, contours=[2], opacity=0.4)
        mlab.pipeline.iso_surface(src, contours=[3], opacity=0.4)
        mlab.pipeline.iso_surface(src, contours=[4], opacity=0.4)
        mlab.pipeline.iso_surface(src_bbox, contours=[1], opacity=0.2)
        # mlab.pipeline.iso_surface(src, contours=[s.max()-0.1*s.ptp(), ],)
        mlab.show()
        count += 1
        if count > 30:
            break
    break